In [ ]:
from __future__ import print_function
import os.path
# import os
import dalmatian as dm
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../../JKBio/')
import TerraFunction as terra
import CCLF_processing
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from IPython.display import Image, display, HTML
import ipdb

# Import requirements for making CNV plots
from matplotlib import pyplot as plt
import cnvlib

In [ ]:
import qgrid # interactive tables
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import gcsfs # to be able to read in files from GCS in Python
import re # used for regex

# Extra options
qgrid.set_grid_option('maxVisibleRows', 10)

# # Show all code cells outputs
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
participant = "PEDS204"

# path would be the participant-specific path
# path = "gs://cclf_results/targeted/neekesh_201912/" 
path = "gs://cclf_results/targeted/neekesh_201912/Wilms_Tumor/PEDS204/"

***
***

# Pretty report generation
After grabbing and making all of the files we want for a given participant (e.g. PEDS182), we want to make a pretty, interactive report. This will be similar to a README except that we will directly embed tables and images. This involves using Jupyter widgets to create dropdown menus and the like. Here are the main functionalities I'd like:

1. kable-like tables that are interactive: sorting, filtering, typing in text or numbers to search, (ability to download sorted/filtered table as a CSV?)
2. ability to quickly go to any image in the directory. I want this so that the user can quickly look through the copy number maps (horizontal plots). Ideally, I'd like to be able to select which one(s) I'd like to view. This could be useful if they want to see two or more at once (i.e. to compare two treatment conditions).

## Automate generation of separate Jupyter notebook for each participant
To do this, we will use Papermill. Papermill automates notebook to notebook generation, and also executes the generated notebook. We may also want to convert the generated notebook to HTML. We can use *nbconvert* for this operation (see https://github.com/jupyter/nbconvert).

In [ ]:
def read_in_tables(filepaths):
    """ takes list of filepaths to TSVs and returns dataframes read in by Pandas"""
    return [pd.read_csv(f, sep='\t', index_col = None) for f in filepaths]

In [ ]:
# A list of file paths for the selected participant
filepaths = ! gsutil ls -r {path}**

# Get all the table filepaths in the bucket
table_filepaths = ! gsutil ls -r {path}*.txt # check: will this search recursively for all .txt files?
to_add = ! gsutil ls -r {path}**.tsv
table_filepaths += to_add

# Get all the png filepaths in the bucket
img_filepaths = ! gsutil ls -r {path}**.png

# Copy all the pngs in the bucket to a tmp folder
# TODO: need to delete the files afterwards
tempdir='../temp/cclfreport/images/'
! gsutil cp -r {path}**.png {tempdir} # copy images from google bucket to local temp folder
# local_img_filepaths = ! ls {tempdir}*.png
local_img_filepaths = [tempdir + os.path.basename(i) for i in img_filepaths]
print(local_img_filepaths)

os.chdir(tempdir)
local_img_file_names = [os.path.basename(i) for i in local_img_filepaths]
os.chdir(cwd)

In [ ]:
def make_interactive_table(filepath, cols_to_include = None):
    """Takes single pd dataframe as input"""
    if type(filepath) != pd.core.frame.DataFrame:
        raise Exception("The function expected a pandas dataframe as input, but got: ", str(type(filepath)))
    data = filepath
    
    # Subset the data to include the specified columns, if any passed in
    if cols_to_include is not None:
        # The index is the first column listed
        index_name = cols_to_include[0]
        data = data[cols_to_include]
        data.set_index(index_name, inplace=True, drop=True)
        if 'keep' in cols_to_include:
            data = data.loc[data['keep'] == True]
            
    # Create and display interactive table
    qgrid_widget = qgrid.show_grid(data, show_toolbar=False, grid_options = {'forceFitColumns': False,
    'defaultColumnWidth': 150})
    display(qgrid_widget)
    print("\n")

# Sample information and identifiers
This section details the external IDs for all the samples we discovered when searching the existing targeted probe data and WES data.

In [ ]:
all_external_ids = ! gsutil ls -r {path}**all_external_ids.tsv
all_failed_external_ids = ! gsutil ls -r {path}**all_failed_external_ids.tsv

# Read in the tables
all_external_ids_df = read_in_tables(all_external_ids)
all_failed_external_ids_df = read_in_tables(all_failed_external_ids)

## Table: all external IDs & associated metadata
The below table is sortable and filterable. You can double-click on the cells in the table if you want to copy the contents, like if you wanted to copy the link to the file in the Google storage console.

In [ ]:
# Make a table with metadata for each external ID

df1 = all_external_ids_df[0]
df1['participant'] = str(os.path.basename(os.path.dirname(all_external_ids[0])))
df1['disease'] = str(os.path.basename(os.path.dirname(os.path.dirname(all_external_ids[0]))))
df1['filepath'] = str(os.path.dirname(all_external_ids[0]))
df1['link'] = str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', os.path.dirname(all_external_ids[0])))

df1.set_index('participant', drop=True, inplace=True)

# Print some summary information
print("We found a total of", df1.shape[0],"external IDs that passed the depth of coverage QC.")
print("Do note that some of these samples may not have been processed yet.")
# TODO: add metadata showing whether we have processed CN, SNV, etc. Ask what other metadata would be useful.

# Display an interactive table
qgrid_widget = qgrid.show_grid(df1, show_toolbar=False, grid_options = {'forceFitColumns': False})
display(qgrid_widget)

## Samples that failed the depth of coverage QC
This summary details all the the external IDs of each sample that failed the depth of coverage QC in the targeted probe pipeline. The depth of coverage QC in the targeted probe pipeline requires that the average gene-level or interval-level coverage is >=50x. 

The summary also lists the participants for which no samples failed the depth of coverage QC.

In [ ]:
# Print the external IDs that passed the QC
failed_qc = False
for table,filepath in zip(all_failed_external_ids_df, all_failed_external_ids):
    tmp_df = table
    participant_name = str(os.path.basename(os.path.dirname(filepath)))
    if tmp_df.shape[0] ==1:
        print("There was", str(tmp_df.shape[0]), "failed sample for participant", participant_name,":")
        display(sorted(tmp_df.iloc[:,0].tolist()))
    elif tmp_df.shape[0] >1:
        print("There were", str(tmp_df.shape[0]), "failed samples for participant", participant_name)
    else:
        failed_qc = True

# If any samples failed the QC, print them:
if failed_qc:
    print("There were no failed samples for participant", participant_name)

# Copy number data

## Copy number heat maps
There are two plots in this section, one for CN data from the targeted probe data and a second for CN data from WES data. To look at any one sample in more detail, you can look either at the corresponding horizontal CN plot in the next section titled "Copy number horizontal plots" or look at the CN table (see either the tables below or the TSV available at the link specified in the "Sample information and identifiers" section.

These tables are searchable and filterable.

### Targeted CN heat map

In [ ]:
# create a heat map specific to the samples requested (either per participant basis or per list of participants) using the plotSomaticCNV workflow in Terra
# Steps: create a new sample set with the appropriate samples, submit a new job, wait for it to finish, copy the picture to the temp dir (and add it to the list of local files), then display it here

### WES CN heat map

In [ ]:
# what's the best way to create a CN heat map for the WES samples? create just using the segmented CN tsv I pull in from Terra? create new workflow?

In [ ]:
cntempdir = '../temp/cclfreport/tables/'
all_cn_tables = wes_cn + tsca_cn
all_cn_tables

In [ ]:
# Create single seg file containing all samples from targeted and WES
local_cn_filepaths = [cntempdir + os.path.basename(i) for i in all_cn_tables]

# Create dict of dicts: {participant_id: {external_id: seg file}}
seg_file_dict = dict()
seg_columns = ["external_id","Chromosome", "Start", "End", "Num_Probes", "Segment_Mean"]

# Read in tables and merge together into one giant seg file
df = pd.concat((pd.read_csv(f,sep="\t", index_col = None) for f in all_cn_tables))
print(set(df.loc[:,"external_id"]))

# Convert to properly-formatted seg file and 
# convert the relative copy ratio to the log2(relative copy ratio)
df = df.loc[:,seg_columns]
df["Segment_Mean"] = np.log2(df["Segment_Mean"])

# Write each external_id level table to a seg file (TSV format)
tmp_path = cntempdir + participant + ".tsv"
df.to_csv(tmp_path, sep = "\t", index = False)

# Convert to .cns format using CNVkit
converted = ! cnvkit.py import-seg {tmp_path} -d {cntempdir}

# Extract .cns filenames
cns_files = []
# TODO: fix pattern so that has backslash before .cns. Otherwise, can end with cns instead of requiring .cns!
pattern = re.compile(r'{}.*.cns'.format(cntempdir))
print("pattern:",pattern)
for file in converted:
    print(file)
    cns_files += [re.search(pattern,file)[0]]
cns_files 

In [ ]:
# cntempdir = '../temp/cclfreport/tables/'
# local_cn_filepaths = [cntempdir + os.path.basename(i) for i in wes_cn]

# # Create dict of dicts: {participant_id: {external_id: seg file}}
# seg_file_dict = dict()
# seg_columns = ["external_id","Chromosome", "Start", "End", "Num_Probes", "Segment_Mean"]
# for filepath in wes_cn:
#     participant_id = str(os.path.basename(os.path.dirname(filepath)))
#     df = pd.read_csv(filepath, sep="\t", index_col = None)
#     df = df.loc[:,seg_columns]
#     print("before:", df[:5])
    
#     # Convert the relative copy ratio to the log2(relative copy ratio)
#     df["Segment_Mean"] = np.log2(df["Segment_Mean"])
#     print("after:", df[:5])

#     # Write each external_id level table to a seg file (TSV format)
#     tmp_path = cntempdir + participant_id + ".tsv"
#     df.to_csv(tmp_path, sep = "\t", index = False)
    
#     # Add key:value pair for the seg files for each participant
#     seg_file_dict[participant_id] = tmp_path

# seg_file_dict

In [ ]:
# cntempdir = '../temp/cclfreport/tables/'
# local_cn_filepaths = [cntempdir + os.path.basename(i) for i in wes_cn]

# for i,gspath in enumerate(wes_cn):
#     # Temporarily download CN seg file locally
#     ! gsutil cp {gspath} {cntempdir}
    
#     # Convert to .cns format
#     ! cnvkit.py import-seg {local_cn_filepaths[i]}
    
# # Remove CN seg files from temporary location
# ! rm {local_cn_filepaths}


In [ ]:
# # Show all code cells outputs
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
%matplotlib inline

# Create CN chromosome plot PDF for each external ID
outfilenames = [cntempdir + os.path.basename(ext_id).split(".")[0] + "-diagram.PDF" for ext_id in cns_files]
for ext_id, segfile in enumerate(segments):    tmp = cnvlib.diagram.create_diagram(cnarr = None, segarr = segfile, threshold = 0 , min_probes =3 , outfname = outfilenames[ext_id])

In [ ]:
# Create CN heat map containing all external IDs for the participant
# https://cnvkit.readthedocs.io/en/stable/plots.html

# TODO: change size of plot dynamically depending on the number of samples to plot
segments = [cnvlib.read(f) for f in cns_files]
plt.rcParams["font.size"] = 32
plt.rcParams['figure.figsize'] = (32,max(6,len(segments)*3))

# plt.figure(figsize=(10,6))
ax = cnvlib.do_heatmap(segments, do_desaturate=False)
ax.set_title("Copy number heatmap for all {} samples".format(participant_id))


plt.tight_layout()
# plt.savefig("test.pdf")

# plt.show()

## Copy number horizontal plots

Select the copy number plot you would like to display from the dropdown menu. The dropdown menu includes CN plots from both targeted probe (TSCA and TWIST) and WES data. The source of the data will be displayed on the title of the image. You can also refer to the table of all external IDs that maps each external ID to the source of the data (see "Sample information and identifiers").

The dropdown menu also includes merged copy number maps for each participant. This PNG file contains all the horizontal CN plots for a given participant in a single place for ease of quick comparison.

**check:** can I add a linked reference to this table so that they can quickly jump there? Might be best to just make it it's own section so that it shows up in the TOC.

<!-- Note that to get nice dropdown menu names, I'm changing directories for now. There's probably a better way to do this. -->

In [ ]:
os.chdir(tempdir)

In [ ]:
# TODO: this isn't the most helpful format / layout currently. Make it possible to select by WES vs Targeted, and select by participant as well. I don't currently know how to do this.
# select image to display from dropdown menu    
@interact
def show_images(file=local_img_file_names):
    print("File name:", file)
    display(Image(file))

In [ ]:
# must change back to the main directory
os.chdir(cwd)

In [ ]:
# Get the CN tables from the Google storage bucket
tsca_cn = ! gsutil ls -r {path}copy_number.tsv
wes_cn = ! gsutil ls -r {path}**wes_copy_number.tsv

# Create dictionary with filepaths as keys and pandas DF as the values
tsca_cn_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in tsca_cn}
wes_cn_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in wes_cn}

# CN columns to display in tables
cn_col_names = ['external_id', 'Sample', 'condition','Chromosome', 'Start', 'End','Segment_Mean', 'Segment_Call', 'Num_Probes']

## Targeted CN table
Select from the dropdown menu to get the targeted CN table for each participant.

In [ ]:
def display_table(file, file_table_dict, cols_to_use):
    # Print key information about the file
    print("Participant: ", str(os.path.basename(os.path.dirname(file))))
    print("Filepath: "+ file)
    print("Link:", str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', file)))
    print("\nTo examine data for a particular external ID, filter the 'external_id' column.")
    
    # Get the TSV from the dict and display it
    df = file_table_dict[file]
    try:
        make_interactive_table(df, cols_to_include = cols_to_use)
    except:
        print("\nAt least one of this participant's mutation file is in a different format from the output of the newest pipeline. This data may be old, and have different column names. No filtering is performed on the displayed table, but you can add additional filters if desired:")
        make_interactive_table(df, cols_to_include = None)

In [ ]:
display_table(file = tsca_cn[0],file_table_dict = tsca_cn_dict, cols_to_use = cn_col_names)

## WES CN table
Select from the dropdown menu to get the WES CN table for each participant, when available. The TSV will contain the data for all the different external IDs.

In [ ]:
display_table(file = wes_cn[0],file_table_dict = wes_cn_dict, cols_to_use = cn_col_names)

## Maybe think about including mutations found in targeted that WEREN'T found in WES. Or, alternatively, just plot a venn diagram. I can only do this for samples where we have both WES and Targeted data - this shouldn't be difficult to figure out.

# Mutation data

Below are interactive tables containing *select* mutation information from the targeted probe data and the WES data. If there were multiple external IDs in either dataset, they have been combined into one table. The external_id column can be used to filter the data so only the mutations for a single external ID is displayed.

Note that this report only includes samples from the targeted data that pass the depth of coverage QC. Samples that did not pass this QC are not included in this report, and their data is not included in the Google bucket. A list of the samples that failed this QC is included earlier in this document (search for "Table: failed QC external IDs").

Also, note that the below tables have been filtered such that the keep column equals True. What this means is that only the variants that passed the filtering steps in the pipeline are included in the tables below. However, the raw mutation TSVs included in the Google bucket contain all the variants regardless of whether keep is True or False if you are interested in that information. This TSV will also contain columns explaining why a mutation was removed during filtration.

Generally speaking, if you are looking for more detailed information about why a mutation you expected to see was filtered out or if you want to get access to all of the columns available in the mutation TSV rather than the ones selected here, you can download the raw mutation TSV from the Google bucket.

In [ ]:
# Get the mutation TSVs
tsca_mut = ! gsutil ls -r {path}mutation.tsv
wes_mut = ! gsutil ls -r {path}**wes_mutations.tsv

# Create dictionary with filepaths as keys and pandas DF as the values
tsca_mut_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in tsca_mut}
wes_mut_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in wes_mut}

## Targeted mutation table

In [ ]:
display_table(file = tsca_mut[0],file_table_dict = tsca_mut_dict, cols_to_use = mut_col_names)

## WES mutation table

In [ ]:
display_table(file = wes_mut[0],file_table_dict = wes_mut_dict, cols_to_use = mut_col_names)